In [ ]:
## https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary

import pandas as pd

In [ ]:
patients = pd.read_csv('patients.csv')
patients.isnull().sum()

In [ ]:
allergies = pd.read_csv('allergies.csv')
allergies.isnull().sum()
#allergies.shape
allergies.head()

In [ ]:
medications = pd.read_csv('medications.csv')
medications.isnull().sum()
#medications.shape

In [ ]:
procedures = pd.read_csv('procedures.csv')
procedures.head()
procedures.isnull().sum()

In [ ]:
conditions = pd.read_csv('conditions.csv')
conditions['PATIENT']

In [ ]:
conditions.head()

In [ ]:
encounter = pd.read_csv('encounters.csv')
encounter.head()

In [ ]:
patients.shape

In [ ]:
1171-1152

In [ ]:
df2 = conditions[conditions['STOP'].isnull()]
df2.head()

In [ ]:
df2.shape

In [ ]:
df2['PATIENT'].nunique()

In [ ]:
1171 - 922

In [ ]:
df3 = conditions[conditions['STOP'].notnull()]
df3.head()

In [ ]:
df3['PATIENT'].nunique()

In [ ]:
1119+922

In [ ]:
allergies.isnull().sum()

In [ ]:
allergies.shape

In [ ]:
597 - 533

From allergies

In [25]:
import pandas as pd
conditions = pd.read_csv('allergies.csv')

In [26]:
total_patients_conditions = conditions['PATIENT'].nunique()
total_patients_non_adherant = conditions[conditions['STOP'].isnull()]['PATIENT'].nunique()
print('Total unique patients in conditions.csv : ',total_patients_conditions)
print('Total unique patients whose condition was not resolved (non - adherent patients) : ',total_patients_non_adherant)
print('Total unique patients whose condition was resolved (adherent patients) : ',total_patients_conditions-total_patients_non_adherant)

Total unique patients in conditions.csv :  141
Total unique patients whose condition was not resolved (non - adherent patients) :  137
Total unique patients whose condition was resolved (adherent patients) :  4


In [27]:
conditions['isadherent'] = conditions['STOP'].notnull()*1
conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,isadherent
0,25-10-1982,NaN,76982e06-f8b8-4509-9ca3-65a99c8650fe,b896bf40-8b72-42b7-b205-142ee3a56b55,300916003,Latex allergy,0
1,25-10-1982,NaN,76982e06-f8b8-4509-9ca3-65a99c8650fe,b896bf40-8b72-42b7-b205-142ee3a56b55,300913006,Shellfish allergy,0
2,25-01-2002,NaN,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,7be1a590-4239-4826-9872-031327f3c368,419474003,Allergy to mould,0
3,25-01-2002,NaN,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,7be1a590-4239-4826-9872-031327f3c368,232347008,Dander (animal) allergy,0
4,25-01-2002,NaN,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,7be1a590-4239-4826-9872-031327f3c368,418689008,Allergy to grass pollen,0


In [28]:
# Extracting patients and isadherent columns seperately
patients_info = conditions[['PATIENT', 'isadherent']]
patients_info.head()

,PATIENT,isadherent
0,76982e06-f8b8-4509-9ca3-65a99c8650fe,0
1,76982e06-f8b8-4509-9ca3-65a99c8650fe,0
2,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,0
3,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,0
4,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,0


In [29]:
#removing duplicates
patients_info = patients_info.drop_duplicates()
patients_info.shape

(160, 2)

In [30]:
print('But total unique patients in conditions.csv : ',total_patients_conditions)

But total unique patients in conditions.csv :  141


In [31]:
group = patients_info.groupby('PATIENT')
patients_info = group.apply(lambda x : x['isadherent'].unique())
patients_info = patients_info.reset_index() 
patients_info.rename(columns = {0:'isadherent'}, inplace = True)

In [32]:
patients_info.head()

,PATIENT,isadherent
0,0288abb6-633c-40c3-ba0c-66c7d957727e,[0]
1,076688b0-f0d5-4c45-8bc6-b206684fa9ac,[0]
2,08ea9043-5f84-46ab-9815-81d90024169a,[0]
3,09616ead-22c8-4210-8cb9-2fdc28e043ca,[0]
4,097079b1-ff8f-4ee0-8ce3-0ea744ecfa21,"[1, 0]"


In [33]:
patients_info1 = group.apply(lambda x : x['isadherent'].nunique())
patients_info1 = patients_info1.reset_index() 
patients_info1.rename(columns = {0:'freq_isadherent'}, inplace = True)
freq_isadherent = patients_info1['freq_isadherent']
patients_info = pd.concat([patients_info, freq_isadherent], axis=1)
patients_info.head()

,PATIENT,isadherent,freq_isadherent
0,0288abb6-633c-40c3-ba0c-66c7d957727e,[0],1
1,076688b0-f0d5-4c45-8bc6-b206684fa9ac,[0],1
2,08ea9043-5f84-46ab-9815-81d90024169a,[0],1
3,09616ead-22c8-4210-8cb9-2fdc28e043ca,[0],1
4,097079b1-ff8f-4ee0-8ce3-0ea744ecfa21,"[1, 0]",2


In [34]:
patients_info.loc[patients_info['freq_isadherent'] > 1, 'isadherent'] = 0
patients_info.drop('freq_isadherent',axis = 1,inplace = True)
print(patients_info['isadherent'].value_counts())

[0]    118
0       19
[1]      4
Name: isadherent, dtype: int64


In [35]:
# converting [0] -> 0 and [1] -> 1
patients_info.loc[patients_info['isadherent'] == 0, 'isadherent'] = 0
patients_info.loc[patients_info['isadherent'] == 1, 'isadherent'] = 1
patients_info['isadherent'].value_counts()

0    137
1      4
Name: isadherent, dtype: int64

In [36]:
patients_info

,PATIENT,isadherent
0,0288abb6-633c-40c3-ba0c-66c7d957727e,0
1,076688b0-f0d5-4c45-8bc6-b206684fa9ac,0
2,08ea9043-5f84-46ab-9815-81d90024169a,0
3,09616ead-22c8-4210-8cb9-2fdc28e043ca,0
4,097079b1-ff8f-4ee0-8ce3-0ea744ecfa21,0
...,...,...
136,f23e12d9-2ec6-4006-b041-ea78d374e9c9,0
137,f36775e8-bf5a-40a2-a223-679c05b46004,0
138,f504b982-e99b-4064-ad25-9e5480e769cd,0
139,f7d7b580-e670-4921-bc67-550ec468d506,0


In [37]:
patients = pd.read_csv('patients.csv')
print('Total shape : ',patients.shape)
patients.info()

Total shape :  (1171, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   1171 non-null   object 
 1   BIRTHDATE            1171 non-null   object 
 2   DEATHDATE            171 non-null    object 
 3   SSN                  1171 non-null   object 
 4   DRIVERS              958 non-null    object 
 5   PASSPORT             898 non-null    object 
 6   PREFIX               927 non-null    object 
 7   FIRST                1171 non-null   object 
 8   LAST                 1171 non-null   object 
 9   SUFFIX               12 non-null     object 
 10  MAIDEN               331 non-null    object 
 11  MARITAL              791 non-null    object 
 12  RACE                 1171 non-null   object 
 13  ETHNICITY            1171 non-null   object 
 14  GENDER               1171 non-null   object 
 15  BIRTHPLACE  

In [38]:
# Droping unnecessary columns
patients.drop(['DEATHDATE','DRIVERS','PASSPORT','PREFIX','SUFFIX','MAIDEN','MARITAL','COUNTY','ZIP','LAT','LON'],axis = 1,inplace = True)
patients.head()

,Id,BIRTHDATE,SSN,FIRST,LAST,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,999-76-6866,José Eduardo181,Gómez206,white,hispanic,M,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,Chicopee,Massachusetts,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,999-73-5361,Milo271,Feil794,white,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,999-27-3385,Jayson808,Fadel536,white,nonhispanic,M,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,999-85-4926,Mariana775,Rutherford999,white,nonhispanic,F,Yarmouth Massachusetts US,999 Kuhn Forge,Lowell,Massachusetts,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,999-60-7372,Gregorio366,Auer97,white,nonhispanic,M,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,598763.07,3772.20


In [39]:
patients_info.rename(columns = {'PATIENT':'Id'}, inplace = True)
patients_info.head()

,Id,isadherent
0,0288abb6-633c-40c3-ba0c-66c7d957727e,0
1,076688b0-f0d5-4c45-8bc6-b206684fa9ac,0
2,08ea9043-5f84-46ab-9815-81d90024169a,0
3,09616ead-22c8-4210-8cb9-2fdc28e043ca,0
4,097079b1-ff8f-4ee0-8ce3-0ea744ecfa21,0


In [40]:
df = (pd.merge(patients, patients_info,how='outer',on = 'Id'))
df.head()

,Id,BIRTHDATE,SSN,FIRST,LAST,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,isadherent
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,999-76-6866,José Eduardo181,Gómez206,white,hispanic,M,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,Chicopee,Massachusetts,271227.08,1334.88,NaN
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,999-73-5361,Milo271,Feil794,white,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,793946.01,3204.49,NaN
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,999-27-3385,Jayson808,Fadel536,white,nonhispanic,M,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,574111.90,2606.40,NaN
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,999-85-4926,Mariana775,Rutherford999,white,nonhispanic,F,Yarmouth Massachusetts US,999 Kuhn Forge,Lowell,Massachusetts,935630.30,8756.19,NaN
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,999-60-7372,Gregorio366,Auer97,white,nonhispanic,M,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,598763.07,3772.20,NaN


In [41]:
df.isnull().sum()

Id                        0
BIRTHDATE                 0
SSN                       0
FIRST                     0
LAST                      0
RACE                      0
ETHNICITY                 0
GENDER                    0
BIRTHPLACE                0
ADDRESS                   0
CITY                      0
STATE                     0
HEALTHCARE_EXPENSES       0
HEALTHCARE_COVERAGE       0
isadherent             1030
dtype: int64

In [42]:
df.shape

(1171, 15)

In [24]:
1171 - 1030

141